# Introduction/Business Problem 

There was described a problem in the beginning of the course: a person needs to relocate and wants to find the same district at the new place.
But it is common when two or more persons participate in the process (a family) and everybody has his/her favorite place.
E.g. he is from Toronto, she is from New York, and they want to find place in Paris.

It is not so easy task because it is not just to find the place close by features to the average place of two.
The goal is to find the place that is the best for all the persons involved.

# Data 

We will use Foursquare data to retrieve place information. Mainly results of explore function because it's free of use.
Plus we will use different functions to merge interests of the persons.